<a href="https://colab.research.google.com/github/iamchenchu/Deep-Learnig-with-TensorFlow/blob/main/08_Food_vision_milestone_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone Project 1 : Food Vision Big


See the annotated vesrion of this notebook on github :    https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb

in the previous notebook (transfer learning part 3: scaling up) we built Food Vision mini: a transfer learning model which beat the original results of the Food101 paper with only 10% of the data.



*   But you might be wondering, what would happen if we used all the data?
*   Well, that's what we're going to find out in this notebook!
*   We're going to be building Food Vision Big™, using all of the data from the Food101 dataset.
* Yep. All 75,750 training images and 25,250 testing images.


**Note :** This time we've got the goal of beating DeepFood, a 2016 paper which used a Convolutional Neural Network trained for 2-3 days to achieve 77.4% top-1 accuracy.






**🔑 Note: Top-1 accuracy means**  :     "accuracy for the top softmax activation value output by the model" (because softmax ouputs a value for every class, but top-1 means only the highest one is evaluated). Top-5 accuracy means "accuracy for the top 5 softmax activation values output by the model", in other words, did the true label appear in the top 5 activation values? Top-5 accuracy scores are usually noticeably higher than top-1.

# Check GPU

Google colab offers free GPUs (thanks to google), however, not all of them are compatable with mixed precision training.

Google colab offers :    
* K80 (Not compatable)
* p100 (Not comapatable)
* Tesla T4 (Compatable)

Knowing this, in order to use mixed precision training we need access to a Tesla T4(from within Google Colab) hardware, our GPU needs a score of more than 7.0+

https://www.tensorflow.org/guide/mixed_precision

https://developer.nvidia.com/cuda-gpus

In [3]:
# If the following line doesn't output "Tesla T4", you can try getting access ti
# Another GPU by going to runtime -> Factory Reset Runtime -> "YES" and then
# re-running this cell.

!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-7a48d752-55c5-a00d-9268-14336545699c)


# Get the helper functions

In the past modules we've created a bunch of the helper functions to do small tasks required for our notebooks.

Rather than rewrite all of these, we can import a script and load them in from there.

The script we've got available can be found on GitHub  :    https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [4]:
# Download helper functions script

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py



--2023-12-24 22:15:54--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-24 22:15:55 (121 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [5]:
# Import series of helper functions for the notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

# TensorFlow Datasets
If you want to get an overview of the TensorFlow Datasets(FTDS), read the below guide :    
https://www.tensorflow.org/api_docs/python/tf/data/Dataset


In [6]:
#Get Tensorflow Datasets
import tensorflow_datasets as tfds



In [10]:
#List all the available datatsets
datasets_list = tfds.list_builders() # Get all the availabel datasets in the TFDS
print(len(datasets_list))
print("food101" in datasets_list)

1261
True


In [11]:
# Load in the datat (takes 5 to 6 miniutes in Google colab)
(train_data, test_data), ds_info =tfds.load(name ="food101",
                                            split=["train", "validation"],
                                            shuffle_files = True,
                                            as_supervised = True, # Data get's returned in tuple format (Data, label)
                                            with_info = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteFKGCYH/food101-train.tfrecord*...:   0%|          …

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteFKGCYH/food101-validation.tfrecord*...:   0%|     …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


# TensorFlow DataSets (TFDS) advantages

Tesorflow datatsets is a place for prepared and ready to use machine learning datasets

**Why do we use the tensorflow datasets :**


*   We can load the data which is already in the tensor format
*   Practice on well established datasets (for many different problem types)
* Experiment with the different modelling techniques on a consistant dataset


**Why not to use the TensorFlow datasets :**  Datasets are static (Do not change like real world datasets)



